In [ ]:
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
train_neg = [tweet[:-1] for tweet in open('./train_neg.txt').readlines()]
train_pos = [tweet[:-1] for tweet in open('./train_pos.txt').readlines()]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_neg + train_pos)

In [ ]:
seq = tokenizer.texts_to_sequences(train_neg + train_pos)

In [ ]:
y = np.array([0]*100000 + [1]*100000)
X = pad_sequences(seq)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)



In [ ]:
max_features=10000
maxlen=x_train.shape[1]
embedding_dim=128
#(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=max_features)

In [ ]:
#x_train=pad_sequences(x_train,maxlen=maxlen)
#x_test=pad_sequences(x_test,maxlen=maxlen)
#print(x_train.shape,x_test.shape)

In [ ]:
num_filters=200 
#
from keras.models import Model
from keras import layers
from keras import Input



text_input=Input(shape=(None,),dtype='int32',name='text')
embedded_text=layers.Embedding(max_features,embedding_dim,input_length=maxlen)(text_input)
x=layers.Bidirectional(layers.LSTM(100,activation='tanh',return_sequences=False, dropout=0.5, recurrent_dropout=0.1))(embedded_text)
x=layers.Dropout(0.5)(x)
x=layers.Dense(30,activation='relu')(x)
output=layers.Dense(1,activation='sigmoid')(x)
model_biLSTM=Model(text_input,output)
model_biLSTM.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text (InputLayer)           [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 128)         1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              183200    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 30)                6030      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                           

In [ ]:
with tf.device('/gpu:0'):
  model_biLSTM.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
  history_biLSTM=model_biLSTM.fit(x_train,y_train,epochs=3,batch_size=64,validation_data=(x_test,y_test))

Epoch 1/3
2813/2813 [==============================] - 1024s 361ms/step - loss: 0.4267 - acc: 0.7955 - val_loss: 0.3799 - val_acc: 0.8220
Epoch 2/3
2813/2813 [==============================] - 1007s 358ms/step - loss: 0.3634 - acc: 0.8327 - val_loss: 0.3694 - val_acc: 0.8316
Epoch 3/3
2813/2813 [==============================] - 1003s 357ms/step - loss: 0.3374 - acc: 0.8471 - val_loss: 0.3662 - val_acc: 0.8339


In [ ]:
test_data = pad_sequences(tokenizer.texts_to_sequences([tweet[:-1] for tweet in open('./test_data.txt').readlines()]))


In [ ]:
prediction = model_biLSTM.predict(test_data)

313/313 [==============================] - 9s 28ms/step


In [ ]:
predictions = (2*(prediction>0.5).astype(int)-1).reshape((10000,)).tolist()

In [ ]:
pd.DataFrame.from_dict({'Id': range(1, 10001), 'Prediction': predictions}).to_csv('submissions_dd.csv', index = False)